In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

2022-11-03 02:45:07.248827: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-03 02:45:07.453566: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-03 02:45:08.307276: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-11-03 02:45:08.307386: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

In [2]:
(train_ds, test_ds), info = tfds.load('mnist',
                                      split=['train', 'test'],
                                      shuffle_files=True,
                                      as_supervised=True,
                                      with_info=True)


2022-11-03 02:45:09.529538: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-11-03 02:45:09.569265: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2022-11-03 02:45:09.569314: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1934] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-11-03 02:45:09.570792: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (o

In [3]:
BATCH_SIZE = 128


def normalize_img(image, label):
    """Normalizes images: `uint8` -> `float32`."""
    return tf.divide(tf.cast(image, tf.float32), 255.), label


train_ds = train_ds.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.cache()
train_ds = train_ds.shuffle(info.splits['train'].num_examples)
train_ds = train_ds.batch(BATCH_SIZE)
train_ds = train_ds.prefetch(tf.data.AUTOTUNE)

test_ds = test_ds.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
test_ds = test_ds.batch(BATCH_SIZE)
test_ds = test_ds.cache()
test_ds = test_ds.prefetch(tf.data.AUTOTUNE)


In [4]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=info.features['image'].shape),  # type: ignore
    tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'),
    tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10),
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

In [5]:
import os

tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=os.path.join('logs', 'mnist'),
    histogram_freq=1,
    write_graph=True,
    write_images=True,
)

model.fit(
    train_ds,
    epochs=10,
    validation_data=test_ds,
    callbacks=[tensorboard_callback],
)


Epoch 1/10
469/469 [==============================] - 34s 63ms/step - loss: 0.2031 - sparse_categorical_accuracy: 0.9414 - val_loss: 0.0621 - val_sparse_categorical_accuracy: 0.9811
Epoch 2/10
469/469 [==============================] - 28s 59ms/step - loss: 0.0597 - sparse_categorical_accuracy: 0.9823 - val_loss: 0.0490 - val_sparse_categorical_accuracy: 0.9852
Epoch 3/10
171/469 [=========>....................] - ETA: 16s - loss: 0.0419 - sparse_categorical_accuracy: 0.9868

KeyboardInterrupt: 